In [1]:
#Length of white part
white_part_length = 4

#Code of white color
white_code = 6

#Valid colors in sequences
color_code_dict = {2: 'blue', 3: 'green', 4: 'yellow', 5: 'red'}
color_nu_dict = {2: 'T', 3: 'A', 4: 'C', 5: 'G'}


#A: Green, C: Yellow, T: blue, G: red


# Convert a string to float number
def to_float(value):
    try:
        v = float(value)
        return v
    except ValueError:
        return -1


#Divide a series of color blocks into chunks.
def find_chunks(colors):
    chunks = []

    start = -1
    current_color = -2

    for i in range(len(colors)):
        if colors[i] != current_color:
            chunks.append({'i': start, 'c': current_color})
            current_color = colors[i]
            start = i
    chunks.append({'i': start, 'c': current_color})
    chunks.append({'i': len(colors) - 1, 'c': -1}) # end of seq marker
    #print(str(chunks))
    return chunks


# Find chunks which distinguish sequences
def find_white_chunks(chunks, times, motor_speed):
    white_chunks = []
    for i in range(len(chunks) - 1):
        if chunks[i]['c'] != white_code: continue
        time_range = times[chunks[i + 1]['i']] - times[chunks[i]['i']]
        distance = time_range * motor_speed
        if round(distance) >= 2:
            white_chunks.append(i)
    return white_chunks


# Map length into block quality. The thresholds are estimated from sequences having a unique color.
def distance2number(distance, color):
    nblocks = int(distance)
    remaining = distance - nblocks

    if color == 3 and remaining >= 0.2:  #Green
        nblocks += 1
    elif color == 5 and remaining >= 0.2:  #Green
        nblocks += 1
    elif color == 4 and remaining >= 0.65:  #Yellow
        nblocks += 1
    elif (color == 2 or color == 6) and remaining >= 0.5:  #Blue
        nblocks += 1
    #print(distance, color, nblocks)
    return nblocks

    '''

    if color == 2: #For blue
        return round(distance/1.09)
    if color == 3: #For green
        return round(distance/0.8)
    if color == 4: #For yellow
        return round(distance/0.89)
    if color == 5: #For red
        return round(distance/0.90)
    return round(distance) #Others
    '''


# Convert value sequence into color sequence
def collect_sequence(chunks, start_index, end_index, read_speed):
    #color_sequence = [''.join(['W' for _ in range(white_part_length)])
    color_sequence = []
    for i in range(start_index + 1, end_index):

        #Skip the chunk with color not in the list (noise)
        color_key = chunks[i]['c']
        if color_key not in color_nu_dict: continue

        # Compute number of blocks
        time_range = times[chunks[i + 1]['i']] - times[chunks[i]['i']]
        distance = time_range * read_speed
        nblocks = distance2number(distance, color_key)

        # Insert to sequence
        color_sequence.extend([chunks[i]['c'] for _ in range(nblocks)])

    #color_sequence.append(''.join(['W' for _ in range(white_part_length)]))
    return color_sequence[::-1]


# Load data file
def read_file(file_name):
    times = []
    colors = []
    with open(file_name, 'r') as file_reader:
        file_reader.readline()
        for line in file_reader:
            strings = line.split(',')
            times.append(to_float(strings[0]))
            colors.append(int(to_float(strings[1])))
    return times, colors


import numpy as np
import matplotlib


#from matplotlib.colors import LinearSegmentedColormap

def draw_sequences(sequences):
    for seq in sequences:

        nletters = len(seq)
        plt.figure(figsize=(nletters, 1))
        plt.xticks([])
        plt.yticks([])
        colors = [color_code_dict[x] for x in seq]
        x = range(nletters)
        plt.bar(x, 1, width=0.98, color=colors)

        letter_seq = [color_nu_dict[x] for x in seq]
        print(''.join(letter_seq))
        for i in x:
            plt.text(i, 0.5, letter_seq[i],
                     horizontalalignment='center',
                     verticalalignment='center',
                     fontsize=20)
    plt.show()


ModuleNotFoundError: No module named 'numpy'

In [ ]:
import matplotlib.pyplot as plt

#Load data
data_file = 'ex01.csv'
times, colors = read_file(data_file)

#Get color chunks
chunks = find_chunks(colors)

#Find the first white chunk (to estimate motor's speed)
start_chunk = -1
for i in range(len(chunks)):
    if chunks[i]['c'] == white_code:
        start_chunk = i
        break
if start_chunk == -1:
    print('Invalid sequence')
    sys.exit()

#Estimate motor's speed
start_duration = times[chunks[start_chunk + 1]['i']] - times[chunks[start_chunk]['i']]
motor_speed = white_part_length / start_duration
white_chunks = find_white_chunks(chunks, times, motor_speed)

#Convert value sequence into color sequence
sequences = []
i = 0
while (i < len(white_chunks) - 1):
    start_index = white_chunks[i]
    end_index = white_chunks[i + 1]

    seq = collect_sequence(chunks, start_index, end_index, motor_speed)
    sequences.append(seq)
    i += 2

plt.plot(times, colors)

draw_sequences(sequences)
